# Исследование надежности заемщиков


В данном исследовании проведен анализ влияния основных факторов на платежеспособность клиента.

Цель: выявить наличие зависимости между семейным положением, количеством детей, уровнем дохода и целью получения кредита и вероятностью просрочки по кредиту. Результаты исследования могут быть использованы для внесения изменений в скоринговую модель банка.

В качестве исходных данных использованы предоставленные клиентом сводные данные по 21,5 тыс. заемщиков. Данные включают в себя сведения о занятости, об уровне дохода, образовании, семейном положении клиента и прочие.

В ходе подготовки к проведению исследования были устранены ошибки в данных, такие как пропуски, выбросы и пр., проведена типизация данных и категоризация по ряду признаков.

В ходе исследования оценим, насколько различные группы клиентов, разбитые по отобранным признакам исправно погашают задолженность, какие группы наиболее подвержены просрочке по платежам.

Выводы приведены в конце работы.

## Импорт библиотек.

In [1]:
import pandas as pd

## Загрузка данных

In [2]:
data = pd.read_csv('data.csv')

## Обзор данных

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Заполним пропуски медианными значениями для соответствующего типа дохода.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Ознакомимся с данными: выведем инфо после всех вышесделанных преобразований, проверим, какие возможные значения принимают величина 'children', величина 'debt', чтобы понять, с чем предстоит работать и выбрать подходящие методы (например, 'debt' в принципе может хранить сведения о сумме просрочки, о количестве дней просрочки и тд, в нашем случае выясняется, что это просто флаг, отмечающий наличие или отсутствие просрочки как таковой).

In [23]:
data.info()
print('Уникальные значения \'children\':', data['children'].unique())
print('Уникальные значения \'debt\':', data['debt'].unique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21331 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21331 non-null  int64  
 1   days_employed          21331 non-null  float64
 2   dob_years              21331 non-null  int64  
 3   education              21331 non-null  object 
 4   education_id           21331 non-null  int64  
 5   family_status          21331 non-null  object 
 6   family_status_id       21331 non-null  int64  
 7   gender                 21331 non-null  object 
 8   income_type            21331 non-null  object 
 9   debt                   21331 non-null  int64  
 10  total_income           21331 non-null  int64  
 11  purpose                21331 non-null  object 
 12  total_income_category  21331 non-null  object 
 13  purpose_category       21331 non-null  object 
dtypes: float64(1), int64(6), object(7)
memory usage: 2.4+ 

Посчитаем общее количество просроченных клиентов и среднее значение просрочки для всей выборки.

In [24]:
print('Всего клиентов:', data['debt'].count(), 'Клиентов с просрочкой:', data['debt'].sum())
print('Средний процент просрочки по всей выборке:', data['debt'].mean() * 100)

Всего клиентов: 21331 Клиентов с просрочкой: 1732
Средний процент просрочки по всей выборке: 8.119638085415593


Теперь сгруппируем выборку по критерию количества детей и посчитаем процент просрочки для каждой группы. Чтобы оставалась возможность проверить самостоятельно точность расчетов, выведем все необходимые промежуточные результаты расчетов на экран, несмотря на явную избыточность этой информации для клиента.

In [25]:
data.groupby('children')['debt'].agg(['count', 'sum', 'mean']).sort_values(by = 'mean', ascending = False)

,count,sum,mean
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4808,444,0.092346
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


**Вывод:** 

Опишем результат: среди бездетных семей процент просрочки ниже среднего, у семей с 1, 2 и 4 детьми процент просрочки выше среднего, при этом обращает внимание, что у семей с 3 детьми просрочка выше среднего, но сильно ниже, чем у семей с 1, 2 и 4 детьми, а у семей с 5 детьми просрочка отсутсвует.
В целом, статистика по России показывает, что рождение детей сказывается на экономическом положении семей негативно, заметно ухудшая их материальное положение. Если взять вышеприведенное утверждение за исходную гипотезу, можем предположить, что наше небольшое исследование гипотезу скорее подтверждает, но с оговорками.
Более хорошая статистика по семьям с 3 и 5 детьми может быть связана с влиянием прочих факторов - например с целью кредита. Так, например, могут влиять особенности законодательного регулирования ипотечного кредитования - на ипотечное жилье не распространяется т.н. "исполнительский иммунитет", что дополнительно дисциплинирует заемщиков по ипотеке.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
data.groupby('family_status')['debt'].agg(['count', 'sum', 'mean'])\
  .sort_values(by = 'mean', ascending = False)

,count,sum,mean
family_status,,,
Не женат / не замужем,2796,273,0.097639
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246


**Вывод:** 

Вдовцы и вдовы платят максимально аккуратно, также процент просрочки ниже среднего у женатых и разведенных, самые ненадежные заемщики неженатые и в гражданском браке

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean'])\
  .sort_values(by = 'mean', ascending = False)

,count,sum,mean
total_income_category,,,
E,22,2,0.090909
C,15921,1353,0.084982
A,25,2,0.080000
B,5014,354,0.070602
D,349,21,0.060172


**Вывод:** 

Явно лучше всех платит вторая категория D, хуже всех категория E

Если опираться на две самых больших категории, можно сделать вывод, что повышение категории дохода ведет к снижению просрочки.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [28]:
data.groupby('purpose_category')['debt'].agg(['count', 'sum', 'mean'])\
  .sort_values(by = 'mean', ascending = False)

,count,sum,mean
purpose_category,,,
операции с автомобилем,4279,400,0.093480
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118
операции с недвижимостью,10751,780,0.072551


**Вывод:** 

Лучше всего с возвратами у кредитов на свадьбу и жилье.

### Шаг 4: общий вывод.

В ходе исследования мы выполнили следующие этапы: предобработка данных, анализ, формулировка выводов.

На этапе предобработки были обработаны пропущенные значения, аномальные значения и дубликаты. Пропущенные значения заменены на медианное значение. Аномальные значения обнаружены в столбцах 'days_imployed' в двух категориях заемщиков: безработные и пенсионеры. Поскольку эти данные в расчета не применяются, какие либо замены не производились. Дубликаты были удалены.

Кроме того, мы сгруппировали всех заемщиков на пять групп в зависимости от размера дохода, где 'A' максимальная категория заемщика (с максимальным доходом), 'E' в свою очередь минимальное значение - клиенты с минимальным уровнем дохода. Это поможет нам исследовать зависимость вероятность просрочки от уровня дохода.
Также цели кредита были приведены к четырем возможным типам. Что также поможет изучить связь целей кредитования с вероятностью просрочки.

Анализ данных заключается в изучении влияния следующих факторов на вероятность возникновения просроченной задолженности: 1) семейное положение клиента, 2) количество детей, 3) уровень дохода, 4) цели получения кредита.
В качестве методики анализа применили анализ средних значений, а именно, нашли среднее значение вероятности просрочки по всей выборке, далее последовательно сгруппировали всю базу заемщиков по четырем вышеописанным критериям и нашли средние значения для каждой группировки. Результаты опишем ниже.

В качестве дальнейших шагов уточнения результатов можем предложить проведение корреляционно - регрессионного анализа, построение многофакторной можели. Применимость предложенных методов требует отдельного изучения.
На этапе изучения зависимости просрочки от количества детей заемщика приходим к выводу, что в целом существует взаимосвязь - рождение детей приводит к росту вероятности просрочки и, в целом рост количества детей также ведет дополнительно к росту вероятности просрочки. При этом были проигнорированы частично выпадающие из общей закономерности значения для заемщиков с 3 и с 5 детьми ввиду малого количества наблюдений и малой статистической значимости этих групп.

На этапе изучения семейного положения пришли к выводу о том, что наличие зарегистрированного брака (включая развод) также положительно влияют на своевременное погашение кредита, а несемейные клиенты менее дисциплинированны и им свойственен рост вероятности прсрочки.
На этапе изучения доходов пришли к выводу, что чем выше доходы клиента, тем выше вероятность своевременного погашения кредита.

Относительно целей получения кредита пришли к выводу, что наиболее рискованными являются кредиты на автомобиль и на образование, в свою очередь кредиты на жилье и на свадьбу.

Согласно проведенному исследованию семейное положение, количество детей, уровень дохода и цели кредита влияют на наличие просроченной задолженности и могут быть учтены в скоринговой модели в целях снижения просроченной задолженности. При этом следует учесть, что внедрение изменений в модель приведет к изменению и других параметров кредитного портфеля.

Отдельно отметим качество исходных данных. Рекомендуем в целях повышения качества данных исключить человеческий фактор в работе с данными - в частности, клиентам предложить заполнять анкету в электронном виде, что позволит избежать возникновения ошибок на этапе ввода данных, также исключить ручную обработку данных сотрудниками банка, и обратить внимание на системы хранения, передачи и обработки данных на предмет возможного возникновения ошибок.